In [ ]:

from gymnasium import spaces, Env
from environment import EnvironmentConfig, create_environment, example_reward_function, example_transition_function
from jax import jit, numpy as jnp
env_width, env_height = 5, 5
action_space_n = 4

dtype = jnp.int32

state_space = spaces.Box(
    low=0, high=max(env_width, env_height), shape=(env_width, env_height), dtype=dtype
)

action_space = spaces.Discrete(action_space_n)

initial_state = jnp.array([0, 0], dtype=dtype)
target_state = jnp.array([4, 4], dtype=dtype)


# Create the environment by creating an EnvironmentConfig object and passing it to the create_environment function

config = EnvironmentConfig(
    state_space=state_space,
    action_space=action_space,
    initial_state=initial_state,
    target_state=target_state,
    reward_function=jit(example_reward_function),
    transition_function=jit(example_transition_function),
)

#--------------------------------------------

import jax
import jax.numpy as jnp
from jax import random, jit

from flax import linen as nn
from flax.training import train_state

import optax

from collections import deque
import numpy as np
import os
import random as rd


# hyperparameters
env = create_environment(config)
state_size = len(state_space.shape)
action_size = env.action_space.n


batch_size = 32  # increase by powers of 2
key = random.PRNGKey(0)
num_episodes = 100  # Number of episodes to simulate
num_iterations = 200  # Number of steps per episode

output_dir = 'results'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)



In [111]:
class DQNModel(nn.Module):
    state_size: int
    action_size: int

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(24)(x)
        x = nn.relu(x)
        x = nn.Dense(24)(x)
        x = nn.relu(x)
        q_values = nn.Dense(self.action_size)(x)
        return q_values
    
class Agent:
    def __init__(self, state_size, action_size):
        super().__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        
        self.key = jax.random.PRNGKey(0)
        self.model = DQNModel(state_size, action_size)
        self.params = self.model.init(self.key, jnp.ones((1, state_size)))
        self.optimizer = optax.adam(self.learning_rate)
        self.state = train_state.TrainState.create(apply_fn=self.model.apply, params=self.params, tx=self.optimizer)
        
        self.model.apply = jit(self.model.apply)
        
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return rd.randrange(self.action_size)
        state = jnp.array(state)
        q_values = self.model.apply(self.state.params, state)
        return jnp.argmax(q_values).item()

    def replay(self, batch_size):
       
        minibatch = rd.sample(self.memory, batch_size)

    



        
        states = jnp.array([experience[0] for experience in minibatch], dtype=jnp.float32)
        actions = jnp.array([experience[1] for experience in minibatch], dtype=jnp.int32)
        rewards = jnp.array([experience[2] for experience in minibatch], dtype=jnp.float32)
        next_states = jnp.array([experience[3] for experience in minibatch], dtype=jnp.float32)
        dones = jnp.array([experience[4] for experience in minibatch], dtype=jnp.bool_)

        # Extract model parameters

        # Compute the target Q-values using JIT compilation
        @jax.jit
        def compute_target_q_values(rewards, gamma, futures, dones):
            return rewards + gamma * futures * (1 - dones) 

        gamma = self.gamma
        futures = jnp.max(self.model.apply(self.state.params, next_states), axis=-1)
        target_q_values = compute_target_q_values(rewards, gamma, futures, dones)


    
                      
        def loss_fn(state, action, target, i):
            q_values = self.model.apply(self.params, state[i])
            q_value = q_values[action[i]]
    
        
            loss = jnp.mean((target[i] - q_value) ** 2)
            return loss
        
        for i in range(batch_size): 
            grads = jax.grad(loss_fn)(states, actions, target_q_values)
            self.state = self.state.apply_gradients(grads=grads)
        
        

        

    def load(self, name):
        self.state = train_state.TrainState.create(
            apply_fn=self.model.apply, params=jnp.load(name), tx=self.optimizer
        )

    def save(self, name):
        jnp.save(name, self.state.params)


# Example usage:
# agent = Agent(state_size=4, action_size=2)
# agent.remember([1, 2, 3, 4], 1, 1, [1, 2, 3, 5], False)
# agent.act([1, 2, 3, 4])
# agent.replay(1)
# agent.save('model_params.npy')
# agent.load('model_params.npy')


In [ ]:
agent = Agent(state_size, action_size)


In [ ]:
#print(f"episode: {episode} state: {state}, reward: {reward}, action: none done: {done}")
#episode_reward = 0  # Track total reward per episode


In [ ]:
done = False
for episode in range(num_episodes):

    state, _, _ = env.reset()
    episode_reward = 0  # Track total reward per episode

    for time in range(num_iterations):
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)

        episode_reward += reward  # Accumulate reward per step
        agent.remember(state, action, reward, next_state, done)

        state = next_state

        if done or time == 199:
            # need better logging
            # add time in 00h 00m 00s then episode number score and loss
            print(
                f"episode: {episode}/{num_episodes} end_state: {state}, score: {episode_reward}, done: {done}"
            )
            break
    
    # Replay the experience
    if len(agent.memory) > 32:
        agent.replay(32)
        if agent.epsilon > agent.epsilon_min:
            agent.epsilon *= agent.epsilon_decay

In [ ]:
STOP

In [ ]:
minibatch = rd.sample(agent.memory, batch_size)
states = jnp.array([experience[0] for experience in minibatch], dtype=jnp.float32)
actions = jnp.array([experience[1] for experience in minibatch], dtype=jnp.int32)
rewards = jnp.array([experience[2] for experience in minibatch], dtype=jnp.float32)
next_states = jnp.array([experience[3] for experience in minibatch], dtype=jnp.float32)
dones = jnp.array([experience[4] for experience in minibatch], dtype=jnp.bool_)


In [ ]:
# Compute the target Q-values using JIT compilation
@jax.jit
def compute_target_q_values(rewards, gamma, futures, dones):
    return rewards + gamma * futures * (1 - dones) 

gamma = agent.gamma
futures = jnp.max(agent.model.apply(agent.state.params, next_states), axis=-1)
target_q_values = compute_target_q_values(rewards, gamma, futures, dones)


In [ ]:
from jax import vmap


In [ ]:

def compute_loss(states, actions, targets, model, params):
    """
    Compute the loss values in parallel using JAX.

    :param states: Array of states
    :param actions: Array of actions
    :param targets: Array of target Q-values
    :param model: A Flax model
    :param params: Parameters of the Flax model
    :return: Array of loss values
    """
    # Vectorize the model function to apply it to all states
    def model_fn(state):
        return model.apply(params, state)
    
    vectorized_model = vmap(model_fn)
    
    # Compute the Q-values for all states
    q_values = vectorized_model(states)
    
    # Select the Q-values corresponding to the actions taken
    q_values = jnp.take_along_axis(q_values, actions[:, None], axis=1).squeeze()
    
    # Compute the squared differences between predicted Q-values and target Q-values
    loss_values = (q_values - targets) ** 2
    
    return loss_values


In [128]:
L = compute_loss(states, actions, target_q_values, agent.model, agent.state.params)
G = loss_fn_batch(states, actions, target_q_values, agent.model, agent.state.params)
L == G

32

In [101]:
#Approach 3: Average all the gradients then just apply one
# In this approach, we compute the gradients for all samples, average them, and then apply the averaged gradient.
def loss_fn_batch(states, actions, targets, model, params):
    def single_loss_fn(state, action, target):
        q_values = model.apply(params, state)
        q_value = q_values[action]
        loss = (q_value - target) ** 2
        return loss
    vectorized_loss_fn = jax.vmap(single_loss_fn, in_axes=(0, 0, 0))
    return vectorized_loss_fn(states, actions, targets)


def update_step_average(states, actions, targets, model, train_state):
    grads = jax.grad(loss_fn_batch)(train_state.params, states, actions, targets, model)
    mean_grads = jax.tree_util.tree_map(lambda x: jnp.mean(x, axis=0), grads)
    train_state = train_state.apply_gradients(grads=mean_grads)
    return train_state

# Example usage
#train_state = update_step_average(states, actions, targets, model, train_state)

In [119]:
mean_grads = jax.tree_util.tree_map(lambda x: jnp.mean(x, axis=0), G)


Array([9.6354194e-02, 1.1718886e+00, 1.0000000e+00, 2.4346778e-01,
       7.1581337e-03, 2.0420419e-01, 1.5995663e+00, 1.0119374e-01,
       3.6279994e-01, 1.5875581e-01, 3.9331135e-01, 1.1718886e+00,
       1.2216524e+01, 2.6676621e+00, 1.1748057e-02, 1.3233642e-01,
       1.4999023e-02, 1.1402357e+00, 1.0934278e+00, 5.4440212e-02,
       1.3233642e-01, 6.3885707e-01, 2.7814272e-01, 2.8563923e-01,
       1.1718886e+00, 8.9648360e-01, 1.8498833e-01, 7.6378258e+01,
       2.7184488e-02, 2.9198304e-01, 5.9655152e-04, 1.3233642e-01],      dtype=float32)

In [135]:
def loss_fn(state, action, target):
    q_values = agent.model.apply(agent.params, state)
    q_value = q_values[action]


    loss = jnp.mean((target - q_value) ** 2)
    return loss
        
h, g = jax.grad(loss_fn)(states[2], actions[2], target_q_values[2])


Array(0., dtype=float32)

Array([ 0.15441827, -0.02721383], dtype=float32)

In [ ]:
#Approach 2: Calculate all the gradients and then apply them all
#In this approach, we compute the gradients for all samples and then apply them all at once.
 
def loss_fn_batch(params, states, actions, targets, model):
    def single_loss_fn(state, action, target):
        q_values = model.apply(params, state)
        q_value = q_values[action]
        loss = (q_value - target) ** 2
        return loss
    vectorized_loss_fn = jax.vmap(single_loss_fn, in_axes=(None, 0, 0, 0))
    return jnp.mean(vectorized_loss_fn(states, actions, targets))

def update_step_batch(states, actions, targets, model, train_state):
    grads = jax.grad(loss_fn_batch)(train_state.params, states, actions, targets, model)
    train_state = train_state.apply_gradients(grads=grads)
    return train_state

# Example usage
train_state = update_step_batch(states, actions, targets, model, train_state)

In [ ]:
# Approach 1: Get gradient, apply it, get the next one, apply it, etc.
# In this approach, we compute and apply the gradient for each sample sequentially.

import jax
from flax.training import train_state

def loss_fn(params, state, action, target, model):
    q_values = model.apply(params, state)
    q_value = q_values[action]
    loss = (q_value - target) ** 2
    return loss

def update_step(state, action, target, model, train_state):
    grads = jax.grad(loss_fn)(train_state.params, state, action, target, model)
    train_state = train_state.apply_gradients(grads=grads)
    return train_state

# Example usage
for state, action, target in zip(states, actions, targets):
    train_state = update_step(state, action, target, model, train_state)